# Bundle Adjustment Example using PyPose and the BAL dataset

```
The dataset is from the following paper:  
Sameer Agarwal, Noah Snavely, Steven M. Seitz, and Richard Szeliski.  
Bundle adjustment in the large.  
In European Conference on Computer Vision (ECCV), 2010.  
```

Link to the dataset: https://grail.cs.washington.edu/projects/bal/

## Install dependencies

In [ ]:
!git clone https://github.com/hxu296/pypose.git
%cd /content/pypose
!git checkout sparse/ba
!pip3 install pypose
!pip install torchdata "portalocker>=2.0.0"
%cd /content/pypose/examples/sparse

: 

## Use dense bundle adjustment solver

In [ ]:
!python ba_dense.py --dataset ladybug --problem problem-49-7776-pre --step 1000

## Visualize loss per observation before and after optimization

In [ ]:
from IPython.display import Image, display

display(Image(filename="loss_per_observation_initial.png"))
display(Image(filename="loss_per_observation_final.png"))